#   Struct

###  Andrea Iskander Belkhir id : 511089

###  Beyza Özdemir id : 576145

In [1]:
using BenchmarkTools
using LinearAlgebra
using StaticArrays 

# S(arg)

it generate (and return)a scaling matrix. the returned matrix have d+1 row and column where d is the lenght of the array passed as arg

In [2]:
function s(args...)
	d = length(args)
	mat = Matrix{Float64}(LinearAlgebra.I, d+1, d+1)
	for k in range(1, length=d)
		mat[k,k]=args[k]
	end
	return mat
end

s (generic function with 1 method)

In [3]:
@btime s(-0.5,-0.5)

  44.242 ns (1 allocation: 160 bytes)


3×3 Matrix{Float64}:
 -0.5   0.0  0.0
  0.0  -0.5  0.0
  0.0   0.0  1.0

In [4]:
@code_llvm s(-0.5,-0.5)

;  @ In[2]:1 within `s'
; Function Attrs: uwtable
define nonnull {}* @julia_s_1803(double %0, double %1) #0 {
pass:
;  @ In[2]:3 within `s'
; ┌ @ C:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.6\LinearAlgebra\src\uniformscaling.jl:471 within `Array'
   %2 = call nonnull {}* @j_Array_1805([1 x i8]* nocapture readonly @_j_const1, [2 x i64]* nocapture readonly @_j_const2) #0
; └
;  @ In[2]:5 within `s'
; ┌ @ array.jl within `setindex!'
   %3 = bitcast {}* %2 to {}**
   %4 = bitcast {}* %2 to double**
   %5 = getelementptr inbounds {}*, {}** %3, i64 3
   %6 = bitcast {}** %5 to i64*
; └
; ┌ @ array.jl:845 within `setindex!'
   %7 = load i64, i64* %6, align 8
   %.not = icmp eq i64 %7, 0
   br i1 %.not, label %oob, label %ib

ib:                                               ; preds = %pass
; └
; ┌ @ array.jl within `setindex!'
   %8 = getelementptr inbounds {}*, {}** %3, i64 4
   %9 = bitcast {}** %8 to i64*
; └
; ┌ @ array.jl:845 within `setindex!'
   %10 = load i64, i64

In [4]:
@benchmark s(-0.5,-0.5)

BenchmarkTools.Trial: 10000 samples with 992 evaluations.
 Range (min … max):  44.456 ns …  1.563 μs  ┊ GC (min … max): 0.00% … 93.36%
 Time  (median):     46.169 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   53.813 ns ± 36.501 ns  ┊ GC (mean ± σ):  3.08% ±  5.02%

  █▇▄▂▁▅▃▁▁▁▁                                 ▁               ▁
  ████████████▇▆▆▆▆▅▆▅▆▅▆▅▆▆▇▆▆▆▅▆▇▆▅▄▄▅▅▄▅▅▆▇█▇▆▆▆▆▅▅▅▃▄▄▃▃▄ █
  44.5 ns      Histogram: log(frequency) by time       141 ns <

 Memory estimate: 160 bytes, allocs estimate: 1.

# Improvment & changes

In [5]:
function s_imp(args...)
	d = length(args)
	mat = MMatrix{d+1,d+1,Float64}(1I)
    @inbounds for k in range(1, length=d)
		mat[k,k]=args[k]
	end
	return mat
end

s_imp (generic function with 1 method)

In [6]:
@btime s_imp(-0.5,-0.5)

  11.411 ns (1 allocation: 80 bytes)


3×3 MMatrix{3, 3, Float64, 9} with indices SOneTo(3)×SOneTo(3):
 -0.5   0.0  0.0
  0.0  -0.5  0.0
  0.0   0.0  1.0

In [7]:
@benchmark s_imp(-0.5,-0.5)

BenchmarkTools.Trial: 10000 samples with 1000 evaluations.
 Range (min … max):  11.400 ns …  3.668 μs  ┊ GC (min … max):  0.00% … 98.00%
 Time  (median):     12.200 ns              ┊ GC (median):     0.00%
 Time  (mean ± σ):   18.123 ns ± 85.325 ns  ┊ GC (mean ± σ):  18.70% ±  4.04%

  █▇▆▃▂▁▁▁▁ ▂▂▁                                   ▁ ▁         ▂
  ███████████████▇▇▆▇▇▆▅▇▆▅▆▅▆▅▅▅▅▅▅▅▄▃▄▅▅▄▁▄▃▁▅▇▆███▇▇▆▇▅▅▆▅ █
  11.4 ns      Histogram: log(frequency) by time      53.9 ns <

 Memory estimate: 80 bytes, allocs estimate: 1.

# Paralelization

In [8]:
function sasyn(args...)
	d = length(args)
	mat = MMatrix{d+1,d+1,Float64}(1I)
    @async for k in range(1, length=d)
		mat[k,k]=args[k]
	end
	return mat
end

sasyn (generic function with 1 method)

## sync and async made the funcion slower

In [9]:
@benchmark sasyn(-0.5,-0.5)

BenchmarkTools.Trial: 4388 samples with 888 evaluations.
 Range (min … max):  435.360 ns …  3.907 ms  ┊ GC (min … max):  0.00% … 99.98%
 Time  (median):     469.932 ns              ┊ GC (median):     0.00%
 Time  (mean ± σ):     2.045 μs ± 63.232 μs  ┊ GC (mean ± σ):  73.41% ±  3.69%

  ▆██▆▅▄▃▃▂▂▁▁▁                                                ▁
  ███████████████▆▇▆▇▆▆▆▆▆▆▇▇▅▅▅▄▅▄▄▄▄▃▁▁▄▅▁▁▄▁▁▁▃▁▅▁▃▁▃▁▁▁▁▃▅ █
  435 ns        Histogram: log(frequency) by time       1.3 μs <

 Memory estimate: 608 bytes, allocs estimate: 6.

In [10]:
function ssimd(args...)
	d = length(args)
	mat = MMatrix{d+1,d+1,Float64}(1I)
    @simd for k in range(1, length=d)
		mat[k,k]=args[k]
	end
	return mat
end

ssimd (generic function with 1 method)

## With the usage of simd we improve the old funcion but we still got worse timing

In [11]:
@benchmark ssimd(-0.5,-0.5)

BenchmarkTools.Trial: 10000 samples with 998 evaluations.
 Range (min … max):  11.523 ns …  2.492 μs  ┊ GC (min … max):  0.00% … 98.54%
 Time  (median):     12.425 ns              ┊ GC (median):     0.00%
 Time  (mean ± σ):   18.334 ns ± 94.354 ns  ┊ GC (mean ± σ):  20.97% ±  4.06%

  ██▇▄▃▂▂ ▁  ▂▂▁▁                                       ▁ ▁▂  ▂
  █████████▇█████▇▇▇▆▆▇▆▆▅▅▆▆▆▅▆▄▅▆▃▅▅▄▅▅▃▄▁▁▃▃▃▁▁▃▁▃▅▇▄████▇ █
  11.5 ns      Histogram: log(frequency) by time        49 ns <

 Memory estimate: 80 bytes, allocs estimate: 1.

# Using threads the funcion is faster then the async but slowen then simd
## (tested with 4 threads)

In [12]:
Threads.nthreads()

4

In [13]:
function sthread(args...)
	d = length(args)
	mat = MMatrix{d+1,d+1,Float64}(1I)
    Threads.@spawn for k in range(1, length=d)
		mat[k,k]=args[k]
	end
	return mat
end

sthread (generic function with 1 method)

In [14]:
@benchmark sthread(-0.5,-0.5)

BenchmarkTools.Trial: 9774 samples with 825 evaluations.
 Range (min … max):  246.182 ns …  11.428 μs  ┊ GC (min … max):  0.00% … 91.34%
 Time  (median):     426.182 ns               ┊ GC (median):     0.00%
 Time  (mean ± σ):   609.925 ns ± 722.553 ns  ┊ GC (mean ± σ):  11.24% ±  9.50%

   █▁                                                            
  ▃██▃▃▄▅▆▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▂ ▂
  246 ns           Histogram: frequency by time         5.37 μs <

 Memory estimate: 622 bytes, allocs estimate: 6.

In [15]:
function get_times(func,N)
    times = Float64[]
    for n in N
        bench = @benchmark $func($(-0.5),$(-0.5))
        push!(times, minimum(bench.times))
    end
    return times
end

get_times (generic function with 1 method)

In [16]:
N = range(1, 4, step=1)
times1 = get_times(s,N)

4-element Vector{Float64}:
 45.5095862764884
 45.5095862764884
 45.5095862764884
 45.483193277310924

In [17]:
times2 = get_times(sasyn,N)


4-element Vector{Float64}:
 411.54285714285714
 388.58447488584477
 399.0990990990991
 398.99497487437185

In [18]:
times3 = get_times(ssimd,N)


4-element Vector{Float64}:
 12.7
 12.612612612612613
 12.751004016064257
 12.2

In [19]:
times4 = get_times(sthread,N)

4-element Vector{Float64}:
 182.5174825174825
 208.5297418630752
 275.0
 227.5189599133261